In [ ]:
import os
if os.getcwd().split('/')[-1] == '_notebooks' or os.getcwd().split('\\')[-1] == '_notebooks':
    os.chdir('..')

import utils
import importlib
importlib.reload(utils)
%load_ext autoreload
%autoreload 2

NUM_IMAGES = 5

COLORIZATION_MODEL = 'diffusers/stable-diffusion-xl-1.0-inpainting-0.1'
UPSCALING_MODEL = 'stabilityai/stable-diffusion-x4-upscaler'
COLORIZATION_NEGATIVE_PROMPT = "black and white, black frame, silhouette, motorbike, toy, clay, model, missing saddle, high saddle, details, detailed, greyscale, duplicate, multiple, detached, shadow, contact shadow, drop shadow, reflection, ground, unrealistic, bad, distorted, ugly, weird"
COLORIZATION_FILL_HOLES = True
COLORIZATION_DILATION = 8
COLORIZATION_STRENGTH = 0.91
COLORIZATION_GUIDANCE = 17
COLORIZATION_MASK_THRESHOLD = 170
COLORIZATION_STEPS = 30

INPAINTING_MASK_THRESHOLD = 200
INPAINTING_MODEL = 'stabilityai/stable-diffusion-2-inpainting'
INPAINTING_NEGATIVE_PROMPT = "wrong proportions, toy, black frame, motorbike, silhouette, model, clay, high saddle, large wheels, text, above floor, flying, changed bike color, white background, duplicate, multiple, people, basket, distortion, low quality, worst, ugly, fuzzy, blurry, cartoon, simple, art"
INPAINTING_GUIDANCE = 15
INPAINTING_STEPS = 75

IMG2IMG_MODEL = 'stabilityai/stable-diffusion-xl-refiner-1.0'
IMG2IMG_STRENGTH = 0.2
IMG2IMG_GUIDANCE = 7.5

In [ ]:
from PIL import Image
bikes = [Image.open('./test_images/for_eval/bikes/sketch/' + x) for x in sorted(os.listdir('./test_images/for_eval/bikes/sketch/'))]
colored_bikes = [Image.open('./test_images/for_eval/bikes/ref/' + x) for x in sorted(os.listdir('./test_images/for_eval/bikes/ref/'))]

with open('./test_images/for_eval/bikes/prompts.txt', 'r') as f:
    prompts = f.readlines()

bike_indices = [int(x.split('--')[0]) for x in prompts]
colorization_prompts = [x.split('--')[1] for x in prompts]
inpainting_prompts = [x.split('--')[2] for x in prompts]

In [ ]:
import matplotlib.pyplot as plt
def selection(imgs: list, step: str) -> Image:
    print('Close the image viewer when you have decided!')

    title = f'Which {step} is best?'

    plt.figure(figsize=(12, 6))
    plt.suptitle(title)

    for i, im in enumerate(imgs):
        plt.subplot(1, len(imgs), i+1)
        plt.title(f'{i+1}')
        plt.axis('off')
        plt.imshow(im)
    plt.tight_layout()
    
    plt.show()

    print()
    selected = input(f'Which {step} is best: ')

    if selected == '':
        return imgs[0]

    selected = int(selected) - 1
    return imgs[selected]

def interactive_generation(bike, col_prompt, in_prompt):
    colorized_imgs = utils.sd_colorization(COLORIZATION_MODEL, UPSCALING_MODEL, bike, col_prompt, COLORIZATION_NEGATIVE_PROMPT, COLORIZATION_MASK_THRESHOLD, COLORIZATION_FILL_HOLES, COLORIZATION_DILATION, COLORIZATION_GUIDANCE, COLORIZATION_STEPS, COLORIZATION_STRENGTH, NUM_IMAGES)
    print(col_prompt)
    colorized_img = selection(colorized_imgs, 'colorization')
    mask = utils.get_mask_from_image(colorized_img, INPAINTING_MASK_THRESHOLD)
    inpainted_imgs = utils.sd_inpainting(INPAINTING_MODEL, colorized_img, mask, in_prompt, INPAINTING_NEGATIVE_PROMPT, INPAINTING_GUIDANCE, INPAINTING_STEPS, NUM_IMAGES)
    print(in_prompt)
    inpainted = selection(inpainted_imgs, 'inpainting')
    results = utils.sd_img2img(IMG2IMG_MODEL, inpainted, in_prompt, INPAINTING_NEGATIVE_PROMPT, IMG2IMG_STRENGTH, IMG2IMG_GUIDANCE, NUM_IMAGES)
    print(in_prompt)
    result = selection(results, 'insertion')
    return result

In [ ]:
os.makedirs('./test_images/eval_results/ours_interactive/', exist_ok=True)

for i, bike in enumerate(bikes):
    display(colored_bikes[i])
    inserted = interactive_generation(bike, colorization_prompts[i], inpainting_prompts[i])
    inserted.save('./test_images/eval_results/ours_interactive/' + inpainting_prompts[i].split(',')[0] + '.png')
    print('Saved!')

products

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

def selection(imgs: list, step: str) -> Image:
    print('Close the image viewer when you have decided!')

    title = f'Which {step} is best?'

    plt.figure(figsize=(12, 6))
    plt.suptitle(title)

    for i, im in enumerate(imgs):
        plt.subplot(1, len(imgs), i+1)
        plt.title(f'{i+1}')
        plt.axis('off')
        plt.imshow(im)
    plt.tight_layout()
    
    plt.show()

    print()
    selected = input(f'Which {step} is best: ')

    if selected == '':
        return imgs[0]

    selected = int(selected) - 1
    return imgs[selected]

def interactive_generation(img, prompt, object_desc):
    try:
        extracted = utils.get_pasted_image(img, object_desc, erosion_strength=4)
        repositioned = utils.paste_pipeline(extracted, 0.9, 0.5, 0.5, rescale=False)
    except:
        print('No erosion!')
        extracted = utils.get_pasted_image(img, object_desc, erosion_strength=0)
        repositioned = utils.paste_pipeline(extracted, 0.9, 0.5, 0.5, rescale=False)
    display(utils.concat_PIL_h(img, repositioned))
    mask = utils.get_mask_from_image(repositioned, INPAINTING_MASK_THRESHOLD)
    inpainted_imgs = utils.sd_inpainting(INPAINTING_MODEL, repositioned, mask, prompt, INPAINTING_NEGATIVE_PROMPT, INPAINTING_GUIDANCE, INPAINTING_STEPS, NUM_IMAGES)
    print(prompt)
    inpainted = selection(inpainted_imgs, 'inpainting')
    results = utils.sd_img2img(IMG2IMG_MODEL, inpainted, prompt, INPAINTING_NEGATIVE_PROMPT, IMG2IMG_STRENGTH, IMG2IMG_GUIDANCE, NUM_IMAGES)
    print(prompt)
    result = selection(results, 'insertion')
    return result

In [ ]:
fp = './test_images/for_eval/products/ref/'
product_filenames = os.listdir('./test_images/for_eval/products/ref/')
product_filenames = [x[1:] for x in product_filenames if x[0] == '_']

prompts = [x.split('.')[0].split('_')[0] for x in product_filenames]
product_descs = [x.split('.')[0].split('_')[1] for x in product_filenames]
product_images = [Image.open(os.path.join(fp, '_' + x)) for x in product_filenames]

In [ ]:
os.makedirs('./test_images/eval_results/ours_interactive/products', exist_ok=True)

for i, product in enumerate(product_images):
    inserted = interactive_generation(product, prompts[i], product_descs[i])
    inserted.save('./test_images/eval_results/ours_interactive/products/' + prompts[i] + '.png')
    print('Saved!')

Cars

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

def selection(imgs: list, step: str) -> Image:
    print('Close the image viewer when you have decided!')

    title = f'Which {step} is best?'

    plt.figure(figsize=(12, 6))
    plt.suptitle(title)

    for i, im in enumerate(imgs):
        plt.subplot(1, len(imgs), i+1)
        plt.title(f'{i+1}')
        plt.axis('off')
        plt.imshow(im)
    plt.tight_layout()
    
    plt.show()

    print()
    selected = input(f'Which {step} is best: ')

    if selected == '':
        return imgs[0]

    selected = int(selected) - 1
    return imgs[selected]

def interactive_generation(img, prompt, object_desc):
    try:
        extracted = utils.get_pasted_image(img, object_desc, erosion_strength=4)
        repositioned = utils.paste_pipeline(extracted, 0.75, 0.5, 0.5, rescale=True)
    except:
        print('No erosion!')
        extracted = utils.get_pasted_image(img, object_desc, erosion_strength=0)
        repositioned = utils.paste_pipeline(extracted, 0.75, 0.5, 0.5, rescale=True)
    display(utils.concat_PIL_h(img, repositioned))
    mask = utils.get_mask_from_image(repositioned, INPAINTING_MASK_THRESHOLD)
    inpainted_imgs = utils.sd_inpainting(INPAINTING_MODEL, repositioned, mask, prompt, INPAINTING_NEGATIVE_PROMPT, INPAINTING_GUIDANCE, INPAINTING_STEPS, NUM_IMAGES)
    print(prompt)
    inpainted = selection(inpainted_imgs, 'inpainting')
    results = utils.sd_img2img(IMG2IMG_MODEL, inpainted, prompt, INPAINTING_NEGATIVE_PROMPT, IMG2IMG_STRENGTH, IMG2IMG_GUIDANCE, NUM_IMAGES)
    print(prompt)
    result = selection(results, 'insertion')
    return result

In [ ]:
from PIL import Image

fp = './test_images/for_eval/cars/ref/'
car_filenames = sorted(os.listdir(fp))

prompts = [x.split('.')[0].split('_')[0] for x in car_filenames]
car_images = [Image.open(os.path.join(fp, '_'+x)) for x in car_filenames]

In [ ]:
os.makedirs('./test_images/eval_results/ours_interactive/', exist_ok=True)

for i, car in enumerate(car_images):
    inserted = interactive_generation(car, prompts[i], 'car')
    inserted.save('./test_images/eval_results/ours_interactive/cars/' + prompts[i] + '.png')
    print('Saved!')